In [1]:
import pandas as pd
import numpy as np

## Read

In [2]:
## read train data
data = pd.read_table('training.txt', sep=' ', header=None)
# name label
data.rename(columns={0:'label'}, inplace=True)
# sepearte labels
label = data.iloc[:, 0]

# remove index: from data
data = data.iloc[:, 1:]
data = data.iloc[:,1:].applymap(lambda x: x[x.find(':')+1:])

# rename columns and merge label
data.columns = list(range(data.shape[1]))
data = data.join(label)
# label: int, other columns: str

In [382]:
## read test data
test = pd.read_table('testing.txt', sep=' ', header=None)

# remove index: from data
test = test.applymap(lambda x: x[x.find(':')+1:])

## Decision Tree

In [ ]:
'''
1.compute the entropy for data-set
2.for every attribute/feature:
       1.calculate entropy for all categorical values
       2.take average information entropy for the current attribute
       3.calculate gain for the current attribute
3. pick the highest gain attribute.
4. Repeat until we get the tree we desired.
'''

In [418]:
# create Tree class
class Tree:
    def __init__(self):
        self.left = None
        self.right = None
        self.data = None
        self.root = None
        self.depth = 0
        # rule of decision tree
        self.rule = None
        # test dataset
        self.test = None
        
    def get_depth(self):
        if self.root == None:
            return self.depth
        else:
            return self.root.get_depth()+1
        
    # set left node
    def set_left(self, left):
        self.left = left
        left.root = self
    
    # set right node
    def set_right(self, right):
        self.right = right
        right.root = self
        
    # set rules
    def set_rule(self, column, feature):
        self.rule = (column, feature)
    
    # set test data
    def set_test(self, test):
        self.test = test
        
    # get left node
    def get_left(self):
        return self.left
    
    # get right node
    def get_right(self):
        return self.right
    
    # get rules
    def get_rules(self):
        return self.rule
        
    # check if leaf
    def is_leaf(self):
        return (self.left is None and self.right is None)
    
    # predict label based on data
    def get_label(self):
        n = self.data.shape[0]
        result = self.data.groupby('label')[0].count()/n
        label = result.idxmax()
        return label

In [315]:
root = Tree()
root.data = data

left = Tree()
left.data = test
root.set_left(left)

In [246]:
def get_gini(data, column, feature):
    n = data.shape[0]
    # counts of each attribute, each class(label) count
    column_count = data.groupby(column)[column].count()
    class_count = data.groupby([column, 'label'])[column].count()

    # gini for left split
    class_exclude = class_count.unstack().drop(index = feature).sum()
    column_exclude = column_count.drop(index=feature).sum()
    prob_exclude = class_exclude/column_exclude
    ratio_exclude = column_exclude/n
    gini_exclude = (1-np.sum((prob_exclude)**2))

    # gini for right split
    class_feature = class_count.unstack().loc[feature]
    column_feature = column_count.loc[feature].sum()
    prob_feature = class_feature/column_feature
    ratio_feature = column_feature/n
    gini_feature = (1-np.sum((prob_feature)**2))

    # combine two gini index
    gini = ratio_exclude*gini_exclude+ratio_feature*gini_feature
    return gini

## Root

In [333]:
%%time
train = data
## calculate gini
n = train.shape[0]
gini_list = []

# loop over columns except labels
for column in train.iloc[:,:-1]:
    for feature in np.unique(train[column]):
        gini = get_gini(train, column, feature)
        gini_list.append((column, feature, gini))

Wall time: 2.24 s


In [334]:
# split tree
column, feature, min_gini = min(gini_list, key=lambda x: x[2])
left = data[data[column]==feature]
right = data[data[column]!=feature]

In [335]:
# tree
root = Tree()
root.rule = (column, feature)

# left
root.left = Tree()
root.left.root = root
root.left.data = left

# right
root.right = Tree()
root.right.root = root
root.right.data = right

## Left

In [336]:
%%time
train = root.left.data
## calculate gini
n = train.shape[0]
gini_list = []

# loop over columns except labels
for column in train.iloc[:,:-1]:
    for feature in np.unique(train[column]):
        gini = get_gini(train, column, feature)
        gini_list.append((column, feature, gini))

Wall time: 1.99 s


In [337]:
# split tree
column, feature, min_gini = min(gini_list, key=lambda x: x[2])
left = train[train[column]==feature]
right = train[train[column]!=feature]

In [338]:
# tree
root.left.rule = (column, feature)

# left
root.left.left = Tree()
root.left.left.root = root.left
root.left.left.data = left

# right
root.left.right = Tree()
root.left.right.root = root.left
root.left.right.data = right

## right

In [339]:
%%time
train = root.right.data
## calculate gini
n = train.shape[0]
gini_list = []

# loop over columns except labels
for column in train.iloc[:,:-1]:
    for feature in np.unique(train[column]):
        gini = get_gini(train, column, feature)
        gini_list.append((column, feature, gini))

Wall time: 2.3 s


In [340]:
# split tree
column, feature, min_gini = min(gini_list, key=lambda x: x[2])
left = train[train[column]==feature]
right = train[train[column]!=feature]

In [341]:
# split tree
column, feature, min_gini = min(gini_list, key=lambda x: x[2])
left = train[train[column]==feature]
right = train[train[column]!=feature]

In [342]:
# tree
root.right.rule = (column, feature)

# left
root.right.left = Tree()
root.right.left.root = root.right
root.right.left.data = left

# right
root.right.right = Tree()
root.right.right.root = root.right
root.right.right.data = right

In [348]:
result.idxmax()

numpy.int64

In [344]:
lln = root.left.left.data.shape[0]
result = root.left.left.data.groupby('label')[column].count()/lln

In [125]:
lrn = root.left.right.data.shape[0]
root.left.right.data.groupby('label')[column].count()/lrn

label
1    0.117063
2    0.277778
3    0.107143
4    0.498016
Name: 55, dtype: float64

In [187]:
rrn = root.right.right.data.shape[0]
root.right.right.data.groupby('label')[column].count()/rrn

label
1    0.307004
2    0.209057
3    0.332280
4    0.151659
Name: 63, dtype: float64

In [188]:
rln = root.right.left.data.shape[0]
root.right.left.data.groupby('label')[column].count()/rln

label
1    0.172632
2    0.296842
3    0.132632
4    0.397895
Name: 63, dtype: float64

## Predicition

In [192]:
#split
column, feature = root.rule
# split tree
left = test[test[column]==feature]
right = test[test[column]!=feature]

In [193]:
column, feature = root.left.rule
# split left tree
ll = left[left[column]==feature]
lr = left[left[column]!=feature]

In [194]:
column, feature = root.right.rule
# split left tree
rl = right[right[column]==feature]
rr = right[right[column]!=feature]

In [195]:
test['label'] = None
test.loc[rl.index, 'label']=int(4)
test.loc[rr.index, 'label']=int(3)
test.loc[ll.index, 'label']=int(2)
test.loc[lr.index, 'label']=int(4)

## Save

In [455]:
with open('result.txt', 'w') as fp:
    [fp.write(str(label)+'\n') for label in  pred]

In [200]:
%%time
train = root.left.left.data
## calculate gini
n = train.shape[0]
gini_list = []

# loop over columns except labels
for column in train.iloc[:,:-1]:
    for feature in np.unique(train[column]):
        gini = get_gini(train, column, feature)
        gini_list.append((column, feature, gini))

Wall time: 2.04 s


## test

In [243]:
def test_split(node):
    train = node.data
    ## calculate gini
    n = train.shape[0]
    gini_list = []

    # loop over columns except labels
    for column in train.iloc[:,:-1]:
        for feature in np.unique(train[column]):
            gini = get_gini(train, column, feature)
            gini_list.append((column, feature, gini))
    
    column, feature, min_gini = min(gini_list, key=lambda x: x[2])
    
    return column, feature, min_gini

In [359]:
def data_split(data, column, feature):
    left_data = data[data[column]==feature]
    right_data = data[data[column]!=feature]
    return left_data, right_data

In [360]:
def get_split(data, column, feature):
    # split data
    left_data, right_data = data_split(data, column, feature)
    
    left = Tree()
    right = Tree()
    left.data =  left_data
    right.data = right_data
    
    return left, right

In [451]:
def predict(root, test):
    # create empty predict
    temp = test.copy()
    temp['predict']= None
    pred = temp['predict']
    
    root.test = test
    divide_list = [root]
    
    # breadth first search
    while divide_list:
        node = divide_list.pop(0)
        print(node.get_depth())
        
        # when node is leaf
        if node.is_leaf():
            # set predicted label
            pred.loc[node.test.index] = node.get_label()
            continue
        
        column, feature = node.get_rules()
        left_data, right_data = data_split(node.test, column, feature)
        
        node.left.set_test(left_data)
        node.right.set_test(right_data)
        
        divide_list.append(node.left)
        divide_list.append(node.right)
    
    return pred

In [447]:
# max_depth, min_node
max_depth = 3
min_node = 200

root = Tree()
root.data = data

search_list = [root]

# breadth first search
while search_list:
    print(len(search_list) )
    node = search_list.pop(0)
    # depth check
    print('node depth: '+str(node.get_depth()))
    if node.get_depth() >= max_depth:
        continue
    
    # get gini based best split
    column, feature, min_gini = test_split(node)
    # divide according to split
    left, right = get_split(node.data, column, feature)
    
    # node number check
    print('left data: '+ str(left.data.shape[0]))
    print('right data: '+ str(right.data.shape[0]))
    
    if left.data.shape[0] <= min_node | right.data.shape[0] <=min_node:
        continue
    
    # set left and right of node
    node.set_left(left)
    node.set_right(right)
    node.set_rule(column, feature)
    
    # append nodes
    search_list.append(left)
    search_list.append(right)

1
node depth: 0
left data: 626
right data: 2374
2
node depth: 1
left data: 122
right data: 504
3
node depth: 1
left data: 475
right data: 1899
4
node depth: 2
left data: 21
right data: 101
5
node depth: 2
left data: 152
right data: 352
6
node depth: 2
left data: 76
right data: 399
7
node depth: 2
left data: 430
right data: 1469
8
node depth: 3
7
node depth: 3
6
node depth: 3
5
node depth: 3
4
node depth: 3
3
node depth: 3
2
node depth: 3
1
node depth: 3


In [454]:
pred

0      3
1      1
2      2
3      1
4      1
5      1
6      1
7      4
8      1
9      1
10     1
11     1
12     1
13     2
14     1
15     1
16     4
17     1
18     1
19     1
20     4
21     1
22     1
23     1
24     1
25     1
26     1
27     1
28     2
29     1
      ..
970    4
971    3
972    1
973    1
974    1
975    1
976    3
977    3
978    1
979    4
980    1
981    1
982    4
983    3
984    1
985    2
986    2
987    1
988    4
989    1
990    1
991    4
992    3
993    1
994    1
995    1
996    1
997    3
998    1
999    1
Name: predict, Length: 1000, dtype: int64